In [1]:
import json
import pandas as pd

In [2]:
df = pd.read_csv(r"word_list\english\unigram_freq.csv")

# only lowercase word
df["word"] = df["word"].apply(lambda x: str(x).lower())
agg_functions = {"word": "first", "count": "sum"}
df = df.groupby(df["word"], as_index=False).aggregate(agg_functions)

# allowed alphabets
df = df[df["word"].str.isalpha()]

# frequent words
df = df[df["count"] > df["count"].quantile(0.75)]

df["word_len"] = df["word"].apply(len)
df = df.sort_values(by=["word"]).reset_index(drop=True)
df.head()

,word,count,word_len
0,a,9081174698,1
1,aa,30523331,2
2,aaa,10243983,3
3,aaaa,1595769,4
4,aaacn,161080,5


In [3]:
df.dtypes

word        object
count        int64
word_len     int64
dtype: object

In [4]:
df.describe()

,count,word_len
count,8.333300e+04,83333.000000
mean,6.937624e+06,7.073380
std,1.324659e+08,2.685928
min,1.365760e+05,1.000000
25%,2.262750e+05,5.000000
50%,4.552740e+05,7.000000
75%,1.492255e+06,9.000000
max,2.313585e+10,32.000000


In [5]:
df.query("count > 10000 and word_len >= 3 and word_len <= 5").describe()

,count,word_len
count,2.437200e+04,24372.000000
mean,1.070548e+07,4.119810
std,1.883967e+08,0.814303
min,1.365770e+05,3.000000
25%,2.304048e+05,3.000000
50%,4.701070e+05,4.000000
75%,1.595851e+06,5.000000
max,2.313585e+10,5.000000


In [6]:
df[(df["word_len"] >= 3) & (df["word_len"] <= 5) & (df["count"] > 10000)].describe()

,count,word_len
count,2.437200e+04,24372.000000
mean,1.070548e+07,4.119810
std,1.883967e+08,0.814303
min,1.365770e+05,3.000000
25%,2.304048e+05,3.000000
50%,4.701070e+05,4.000000
75%,1.595851e+06,5.000000
max,2.313585e+10,5.000000


In [7]:
df.query("'running' in word")

,word,count,word_len
64050,running,72879426,7


In [13]:
difficulties = {"easy": 0.95, "medium": 0.9, "hard": 0.7, "all": 0}
min_len = 4
max_len = 5

for name, base_percentile in difficulties.items():
    dfs_filtered: list[pd.DataFrame] = []
    lengths: list[str] = []
    for l in range(min_len, max_len + 1):
        df_len = df[df["word_len"] == l]
        # longer words need larger vocab pool
        percentile = max(base_percentile - (l - min_len) * 0.07, 0)
        cutoff = df_len["count"].quantile(percentile)
        dfs_filtered.append(
            df_len[df_len["count"] > cutoff]
            .sort_values(by=["count"], ascending=False)
            .reset_index(drop=True)
        )
        lengths.append(f"len{l}: #{len(dfs_filtered[-1])}")
    df_filtered = pd.concat(dfs_filtered)

    print(", ".join(lengths))

    with open(rf"vocabs\en\en-{name}.json", "w") as f:
        data = df_filtered["word"].sort_values().tolist()
        json.dump(data, f)
        print(f"{f.name}: {len(df_filtered)} words")

len4: #394, len5: #1166
vocabs\en\en-easy.json: 1560 words
len4: #787, len5: #1652
vocabs\en\en-medium.json: 2439 words
len4: #2359, len5: #3595
vocabs\en\en-hard.json: 5954 words
len4: #7861, len5: #9714
vocabs\en\en-all.json: 17575 words
